Zad 1/2

In [7]:
import inspect
from enum import Enum


class Field:

    class FieldType(Enum):
        INTEGER = 1
        FLOAT = 2
        STRING = 3
        DATE = 4

    def __init__(self, field_type: 'Field.FieldType'):
        if not isinstance(field_type, Field.FieldType):
            raise ValueError("field_type must be an instance of Field.FieldType")
        self.field_type = field_type

    def get_fieldtype(self):
        return self.field_type

    def __str__(self):
        return self.field_type.__class__.__name__


class Model:

    def __init__(self, db_table=None):
        self.db_table = self.generate_table_for_name(self.__class__.__name__) if db_table is None else db_table

    def get_fields(self):
        fields = {}
        for name, obj in inspect.getmembers(self):
            if isinstance(obj, Field):
                fields[name] = obj.get_fieldtype()
        return fields

    def __setattr__(self, attr, val):
        for name, obj in inspect.getmembers(self):
            if name == attr and isinstance(obj, Field):
                obj.value = val
                return
        super().__setattr__(attr, val)

    @staticmethod
    def generate_table_for_name(name: str):
        return f'db_{name.lower()}'

    @classmethod
    def from_dict(cls, name: str, fields: dict[str, Field]):
        for field in fields.items():
            match field:
                case (str(), Field()):
                    setattr(cls, field[0], field[1])

        model = cls()
        model.db_table = cls.generate_table_for_name(name)
        return model

class Person(Model):
    id = Field(Field.FieldType.INTEGER)
    firstname = Field(Field.FieldType.STRING)
    lastname = Field(Field.FieldType.STRING)
    age = Field(Field.FieldType.INTEGER)

In [14]:
person = Person()
print(person.db_table)
print(person.get_fields())

assert isinstance(Person.id, Field)
assert isinstance(Person.id.field_type, Field.FieldType)

new_field = Field(Field.FieldType.DATE)
print(new_field.get_fieldtype())  #

db_person
{'age': <FieldType.INTEGER: 1>, 'firstname': <FieldType.STRING: 3>, 'id': <FieldType.INTEGER: 1>, 'lastname': <FieldType.STRING: 3>}
FieldType.DATE


Zad 3

In [17]:
import inspect
from enum import Enum


class Field:

    class FieldType(Enum):
        INTEGER = 1
        FLOAT = 2
        STRING = 3
        DATE = 4

    def __init__(self, field_type: 'Field.FieldType'):
        if not isinstance(field_type, Field.FieldType):
            raise ValueError("field_type must be an instance of Field.FieldType")
        self.field_type = field_type
        self._value = None

    def _get_field_value(self):
        return self._value

    def _set_field_value(self, value):
        self._value = value

    def get_fieldtype(self):
        return self.field_type

    def __str__(self):
        return self.field_type.__class__.__name__


class StringField(Field):
    def __init__(self):
        super().__init__(Field.FieldType.STRING)

    def _set_field_value(self, value):
        if not isinstance(value, str):
            raise ValueError("Wartość pola StringField musi być typu string")
        super()._set_field_value(value)


class Model:

    def __init__(self, db_table=None):
        self.db_table = self.generate_table_for_name(self.__class__.__name__) if db_table is None else db_table

    def get_fields(self):
        fields = {}
        for name, obj in inspect.getmembers(self):
            if isinstance(obj, Field):
                fields[name] = obj.get_fieldtype()
        return fields

    def __setattr__(self, attr, val):
        for name, obj in inspect.getmembers(self):
            if name == attr and isinstance(obj, Field):
                obj._set_field_value(val)
                return
        super().__setattr__(attr, val)

    @staticmethod
    def generate_table_for_name(name: str):
        return f'db_{name.lower()}'

    @classmethod
    def from_dict(cls, name: str, fields: dict[str, Field]):
        for field in fields.items():
            match field:
                case (str(), Field()):
                    setattr(cls, field[0], field[1])

        model = cls()
        model.db_table = cls.generate_table_for_name(name)
        return model

class Person(Model):
    id = Field(Field.FieldType.INTEGER)
    firstname = StringField()
    lastname = StringField()
    age = Field(Field.FieldType.INTEGER)

person = Person()
person.firstname = "Jakub"
person.lastname = "Karcz"

print(person.firstname._get_field_value())
print(person.lastname._get_field_value())


try:
    person.firstname = 123
except ValueError as e:
    print(e)
print(person.get_fields())


Jakub
Karcz
Wartość pola StringField musi być typu string
{'age': <FieldType.INTEGER: 1>, 'firstname': <FieldType.STRING: 3>, 'id': <FieldType.INTEGER: 1>, 'lastname': <FieldType.STRING: 3>}


Zad 4


In [16]:
import inspect
from enum import Enum


class Field:

    class FieldType(Enum):
        INTEGER = 1
        FLOAT = 2
        STRING = 3
        DATE = 4

    def __init__(self, field_type: 'Field.FieldType'):
        if not isinstance(field_type, Field.FieldType):
            raise ValueError("field_type must be an instance of Field.FieldType")
        self.field_type = field_type
        self._values = {}

    def _get_field_value(self, instance):
        return self._values.get(instance, None)

    def _set_field_value(self, instance, value):
        self._values[instance] = value

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self._get_field_value(instance)

    def __set__(self, instance, value):
        self._set_field_value(instance, value)

    def get_fieldtype(self):
        return self.field_type

    def __str__(self):
        return f"{self.field_type.name}"


class StringField(Field):
    def __init__(self):
        super().__init__(Field.FieldType.STRING)

    def _set_field_value(self, instance, value):
        if not isinstance(value, str):
            raise ValueError("Wartość pola StringField musi być typu string")
        super()._set_field_value(instance, value)


class Model:

    def __init__(self, db_table=None):
        self.db_table = self.generate_table_for_name(self.__class__.__name__) if db_table is None else db_table

    def get_fields(self):
        fields = {}
        for name, obj in inspect.getmembers(self.__class__):
            if isinstance(obj, Field):
                fields[name] = obj.get_fieldtype()
        return fields

    @staticmethod
    def generate_table_for_name(name: str):
        return f'db_{name.lower()}'

    @classmethod
    def from_dict(cls, name: str, fields: dict[str, Field]):
        for field in fields.items():
            match field:
                case (str(), Field()):
                    setattr(cls, field[0], field[1])

        model = cls()
        model.db_table = cls.generate_table_for_name(name)
        return model

class Movie(Model):
    title = StringField()
    director = StringField()

movie = Movie()
movie.title = "Alice in the Wonderland"
movie.director = "Ktoś taki"

print(movie.title)
print(movie.director)
print(movie.get_fields())


Alice in the Wonderland
Ktoś taki
{'director': <FieldType.STRING: 3>, 'title': <FieldType.STRING: 3>}
